# Decompositions Playground

Given an integer number, $target$, we want to get all the possible combinations of at most $k$ smaller _standard_ numbers that sum up to the target number. However this problem is just a special case of the problem of finding all the possible combinations of a set of given numbers so, let start calculating the universe of all the possible combinations.

# The Algorithm

Starting with an empty set ${ }$, we use it as a template to be extended by each of the _standard denominations_ smaller or equal than the tail element of the template set and as result we obtains a new set of template sets. For example:

```
take {} and extend:
{10}
{9}
{8}
{6}
... (continue)
```

Then we repeat the process for each new template set, this means, take a template set and extend it with each of the _standard denominations_. For example:

```
take {10} and extend
{10, 10}
{10, 9}
{10, 8}
{10, 6}
.... (continue)
take {9} and extend
{9, 9}
{9, 8}
{9, 6}
... (and so on)
```

Basically it is a recursive process that builds the new template sets by appending each of the _standard denominations_, smaller or equal than the tail element of the template set, to each of the template sets.

# Implementation

Lets calculate the the decompositions of $k = 3$ and $std_denominations = [1, 2, 3, 4, 5, 6, 8, 9, 10]$

In [ ]:
using System.Collections.Immutable;

// Parameters
var denoms = new long[] { 10, 9, 8, 6, 5, 4, 3, 2, 1 };
var initialState = ImmutableList<long>.Empty;
var k = 3;  // the maximum number of elements in a decomposition.

// Other parameters we will use much later
var target = 21;
var tolerance = 1;

But before start let create a helper method to print all the decompositions. This will help us to reduce the noise in our code (c# is already too noisy for this kind of tasks)

In [ ]:
void Print(IEnumerable<ImmutableList<long>> decompositions)
{
	foreach (var composition in decompositions)
		Console.WriteLine("Sum: {0} -> [{1}]", composition.Sum(), string.Join(", ", composition));	
}

And this is the *code*:

In [ ]:
IEnumerable<ImmutableList<long>> Combinations1(long currentDenomination, ImmutableList<long> accumulator, int k, long[] denoms) =>
	k == 0
		? new[] { accumulator.Add(currentDenomination) }
		: denoms.SelectMany((std, i) => Combinations1(std, accumulator.Add(currentDenomination), k - 1, denoms[i..]));

Print(denoms.SelectMany((std, i) => Combinations1(std, initialState, k - 1, denoms[i..])));

Sum: 30 -> [10, 10, 10]
Sum: 29 -> [10, 10, 9]
Sum: 28 -> [10, 10, 8]
Sum: 26 -> [10, 10, 6]
Sum: 25 -> [10, 10, 5]
Sum: 24 -> [10, 10, 4]
Sum: 23 -> [10, 10, 3]
Sum: 22 -> [10, 10, 2]
Sum: 21 -> [10, 10, 1]
Sum: 28 -> [10, 9, 9]
Sum: 27 -> [10, 9, 8]
Sum: 25 -> [10, 9, 6]
Sum: 24 -> [10, 9, 5]
Sum: 23 -> [10, 9, 4]
Sum: 22 -> [10, 9, 3]
Sum: 21 -> [10, 9, 2]
Sum: 20 -> [10, 9, 1]
Sum: 26 -> [10, 8, 8]
Sum: 24 -> [10, 8, 6]
Sum: 23 -> [10, 8, 5]
Sum: 22 -> [10, 8, 4]
Sum: 21 -> [10, 8, 3]
Sum: 20 -> [10, 8, 2]
Sum: 19 -> [10, 8, 1]
Sum: 22 -> [10, 6, 6]
Sum: 21 -> [10, 6, 5]
Sum: 20 -> [10, 6, 4]
Sum: 19 -> [10, 6, 3]
Sum: 18 -> [10, 6, 2]
Sum: 17 -> [10, 6, 1]
Sum: 20 -> [10, 5, 5]
Sum: 19 -> [10, 5, 4]
Sum: 18 -> [10, 5, 3]
Sum: 17 -> [10, 5, 2]
Sum: 16 -> [10, 5, 1]
Sum: 18 -> [10, 4, 4]
Sum: 17 -> [10, 4, 3]
Sum: 16 -> [10, 4, 2]
Sum: 15 -> [10, 4, 1]
Sum: 16 -> [10, 3, 3]
Sum: 15 -> [10, 3, 2]
Sum: 14 -> [10, 3, 1]
Sum: 14 -> [10, 2, 2]
Sum: 13 -> [10, 2, 1]
Sum: 12 -> [10, 1, 1]


# Compute only those sets that sum up to a target value (or less)

The trick here is to limit the set of _standard denominations_ the algorithm can use in each iteration. For example, imagine the target is $25$:

| target | set            | sum  | remaining | standard denominations available |
|-------:|---------------:|-----:|----------:|---------------------------------:|
|     25 |         { 10 } |   10 |        15 |  { 10, 9 , 8, 6, 5, 4, 3, 2, 1 } |
|     15 |     { 10, 10 } |   20 |         5 |                { 5, 4, 3, 2, 1 } |
|      5 | { 10, 10,  5 } |   25 |         0 |                              { } |

Below we have the code that filters those values that are too big to be explored.

In [ ]:
IEnumerable<ImmutableList<long>> CombinationsUpto(long currentDenomination, long target, ImmutableList<long> accumulator, int k, long[] denoms)
{
	accumulator = accumulator.Add(currentDenomination);
	var remaining = target - currentDenomination;
	if (k == 0)
		return new[] { accumulator };
	
	denoms = denoms.SkipWhile(x => x > remaining).ToArray();
	return denoms.SelectMany((std, i) => CombinationsUpto(std, remaining, accumulator, k - 1, denoms[i..]));
}

Print(denoms.SelectMany((std, i) => CombinationsUpto(std, target, initialState, k - 1, denoms[i..])));

Sum: 21 -> [10, 10, 1]
Sum: 21 -> [10, 9, 2]
Sum: 20 -> [10, 9, 1]
Sum: 21 -> [10, 8, 3]
Sum: 20 -> [10, 8, 2]
Sum: 19 -> [10, 8, 1]
Sum: 21 -> [10, 6, 5]
Sum: 20 -> [10, 6, 4]
Sum: 19 -> [10, 6, 3]
Sum: 18 -> [10, 6, 2]
Sum: 17 -> [10, 6, 1]
Sum: 20 -> [10, 5, 5]
Sum: 19 -> [10, 5, 4]
Sum: 18 -> [10, 5, 3]
Sum: 17 -> [10, 5, 2]
Sum: 16 -> [10, 5, 1]
Sum: 18 -> [10, 4, 4]
Sum: 17 -> [10, 4, 3]
Sum: 16 -> [10, 4, 2]
Sum: 15 -> [10, 4, 1]
Sum: 16 -> [10, 3, 3]
Sum: 15 -> [10, 3, 2]
Sum: 14 -> [10, 3, 1]
Sum: 14 -> [10, 2, 2]
Sum: 13 -> [10, 2, 1]
Sum: 12 -> [10, 1, 1]
Sum: 21 -> [9, 9, 3]
Sum: 20 -> [9, 9, 2]
Sum: 19 -> [9, 9, 1]
Sum: 21 -> [9, 8, 4]
Sum: 20 -> [9, 8, 3]
Sum: 19 -> [9, 8, 2]
Sum: 18 -> [9, 8, 1]
Sum: 21 -> [9, 6, 6]
Sum: 20 -> [9, 6, 5]
Sum: 19 -> [9, 6, 4]
Sum: 18 -> [9, 6, 3]
Sum: 17 -> [9, 6, 2]
Sum: 16 -> [9, 6, 1]
Sum: 19 -> [9, 5, 5]
Sum: 18 -> [9, 5, 4]
Sum: 17 -> [9, 5, 3]
Sum: 16 -> [9, 5, 2]
Sum: 15 -> [9, 5, 1]
Sum: 17 -> [9, 4, 4]
Sum: 16 -> [9, 4, 3]
Sum: 15

This is better because it doesn't generate lots and lots of unnecessary combinations but it still has one problems and it is that it still generates lots and lots of useless combinations because their sum is too far from the expected target. It is important to abort the generation of those combinations once we know they are not possible.

For example, imagine we want to generate decompositions of $k=3$ that sum up to 12 using our previously define _standard denominations_. Imagine we are generating the first set:

| target | set            | sum  | remaining | denominations available | can continue? |
|-------:|---------------:|-----:|----------:|---------------------------------:|---------------
|     12 |         { 10 } |   10 |         2 |  { 2, 1 } | Yes. Because there are space for 2 extra elements and 2 * 2 >= 2 (remaining). |
|     12 |     {  6,  3 } |    9 |         3 |  { 3, 2, 1 } | Yes. Because there is space for 1 extra elements and 1 * 3 >= 3 (remaining).
|     12 |     {  5,  3 } |    8 |         4 |  { 3, 2, 1 } | No. Because there is space for 1 extra elements and 1 * 3 < 4 (remaining).

In [ ]:
IEnumerable<ImmutableList<long>> CombinationsUptoAndPrune(long currentDenomination, long target, ImmutableList<long> accumulator, int k, long[] denoms)
{
	accumulator = accumulator.Add(currentDenomination);
	var remaining = target - currentDenomination;
	if (k == 0)
		return new[] { accumulator };
	
	denoms = denoms.SkipWhile(x => x > remaining).ToArray();
	return denoms
		.TakeWhile(std => k * std >= remaining)
		.SelectMany((std, i) => CombinationsUptoAndPrune(std, remaining, accumulator, k - 1, denoms[i..]));
}

Print(denoms.SelectMany((std, i) => CombinationsUptoAndPrune(std, target, initialState, k - 1, denoms[i..])));

Sum: 21 -> [10, 10, 1]
Sum: 21 -> [10, 9, 2]
Sum: 21 -> [10, 8, 3]
Sum: 21 -> [10, 6, 5]
Sum: 21 -> [9, 9, 3]
Sum: 21 -> [9, 8, 4]
Sum: 21 -> [9, 6, 6]
Sum: 21 -> [8, 8, 5]


This is much much better because now we get the only 8 decompositions that sum up to 21. However, not all numbers can be expressed as the sum of other smaller positive numbers. Imagine our _standard denominations_ are just ${ 100, 50, 25, 10, 5 }$, in this case it is impossible to represent the number $21$.

For this reason we need to introduce the concept of _tolerance_, defined as the acceptable difference between the expected target and the one obteined as the result of the sum of the combination individual elements.

In [ ]:
IEnumerable<ImmutableList<long>> CombinationsUptoAndPruneWithTolerance(long currentDenomination, long target, long tolerance, ImmutableList<long> accumulator, int k, long[] denoms)
{
	accumulator = accumulator.Add(currentDenomination);
	var remaining = target - currentDenomination;
	if (k == 0 || remaining < tolerance)
		return new[] { accumulator };
	
	denoms = denoms.Where(x => x <= remaining).ToArray();
	return denoms
		.TakeWhile(std => k * std >= remaining - tolerance)
		.SelectMany((std, i) => CombinationsUptoAndPruneWithTolerance(std, remaining, tolerance, accumulator, k - 1, denoms[i..]));
}

Print(denoms.SelectMany((std, i) => CombinationsUptoAndPruneWithTolerance(std, target, tolerance, initialState, k - 1, denoms[i..])));

Sum: 21 -> [10, 10, 1]
Sum: 21 -> [10, 9, 2]
Sum: 20 -> [10, 9, 1]
Sum: 21 -> [10, 8, 3]
Sum: 20 -> [10, 8, 2]
Sum: 21 -> [10, 6, 5]
Sum: 20 -> [10, 6, 4]
Sum: 20 -> [10, 5, 5]
Sum: 21 -> [9, 9, 3]
Sum: 20 -> [9, 9, 2]
Sum: 21 -> [9, 8, 4]
Sum: 20 -> [9, 8, 3]
Sum: 21 -> [9, 6, 6]
Sum: 20 -> [9, 6, 5]
Sum: 21 -> [8, 8, 5]
Sum: 20 -> [8, 8, 4]
Sum: 20 -> [8, 6, 6]


# Do not explore further once a much is find

This version looks better, right? But if take a look at the 3rd result ($[10, 9, 1]$ = 20) we should ask why!? What's the point of that combination? Clearly if we explore the combinations in expanding the template (accumulator) with standard denominations in descending order then, it is silly to continue searching for combinations once we have already found one exact much because all the subsecuent combinations will sum less.

We will need a linq extension to make the code look easier. `TakeUnti` is a method that takes a sequence and returns a sequence of elements until a condition is met with the first element that meets the condition included.

In [ ]:
static IEnumerable<T> TakeUntil<T>(this IEnumerable<T> source, Func<T, bool> predicate)
{
	foreach(T el in source)
	{
		yield return el;
		if (predicate(el))
			yield break;
	}
}

In [ ]:
IEnumerable<ImmutableList<long>> CombinationsUptoAndPruneWithTolerance2(long currentDenomination, long target, long localTarget, long tolerance, ImmutableList<long> accumulator, int k, long[] denoms)
{
	accumulator = accumulator.Add(currentDenomination);
	var remaining = localTarget - currentDenomination;
	if (k == 0 || remaining < tolerance)
		return new[] { accumulator };
	
	denoms = denoms.Where(x => x <= remaining).ToArray();
	return denoms
		.TakeWhile(std => k * std >= remaining - tolerance)
		.SelectMany((std, i) => 
			CombinationsUptoAndPruneWithTolerance2(std, target, remaining, tolerance, accumulator, k - 1, denoms[i..])
			.TakeUntil(x => x.Sum() == target));
}

Print(denoms.SelectMany((std, i) => CombinationsUptoAndPruneWithTolerance2(std, target, target, tolerance, initialState, k - 1, denoms[i..])));

Sum: 21 -> [10, 10, 1]
Sum: 21 -> [10, 9, 2]
Sum: 21 -> [10, 8, 3]
Sum: 21 -> [10, 6, 5]
Sum: 20 -> [10, 5, 5]
Sum: 21 -> [9, 9, 3]
Sum: 21 -> [9, 8, 4]
Sum: 21 -> [9, 6, 6]
Sum: 21 -> [8, 8, 5]
Sum: 20 -> [8, 6, 6]


# Optimization (first part)

In all previous code snippets we have used an `ImmutableList` as accumulator bacause it makes an efficient usage of memory given it is implemented with Avl trees. However, we want to reduce the heap allocations and total memory usage as much as possible what is doable if we introduce some assumptions:

* $k$ will never be greater than $8$ and,
* $len(standarddenoms)$ will never be larger than $2^8$

With this we can use an `ulong` to store the indexes of the _standard denomination_ instead of their values. 



In [ ]:
IEnumerable<(long Sum, ulong Decomposition)> CompactCombinationsOfIndexesUptoAndPruneWithTolerance(int currentDenominationIdx, long target, long localTarget, long tolerance, ulong accumulator, long sum, int k, long[] denoms)
{
	accumulator = (accumulator << 8) | ((ulong)currentDenominationIdx & 0xff);
	var currentDenomination = denoms[currentDenominationIdx];
	sum += currentDenomination;
	var remaining = localTarget - currentDenomination;
	if (k == 0 || remaining < tolerance)
		return new[] { (sum, accumulator) };

	currentDenominationIdx += denoms.Skip(currentDenominationIdx).Count(x => x > remaining);
	return Enumerable.Range(0, denoms.Length - currentDenominationIdx)
		.TakeWhile(i => k * denoms[currentDenominationIdx + i] >= remaining - tolerance)
		.SelectMany((_, i) => 
			CompactCombinationsOfIndexesUptoAndPruneWithTolerance(currentDenominationIdx + i, target, remaining, tolerance, accumulator, sum, k - 1, denoms)
			.TakeUntil(x => x.Sum == target));
}



In [ ]:
void Print(IEnumerable<(long Sum, ulong Decomposition)> decompositions)
{
	foreach (var (composition, row) in decompositions.Select((x,r) => (x, r)))
	{
		var indexes = BitConverter.GetBytes(composition.Decomposition);
		var remaining = composition.Sum;
		var i = 0;
		var list = new List<long>();
		do
		{
			var val = denoms[indexes[i++]];
			list.Insert(0, val);
			remaining -= val;
		}while(remaining > 0);
		Console.WriteLine("#{0} Sum: {1} -> [{2}]", row, composition.Sum, string.Join(", ", list));
	}
}

In [ ]:
Print(denoms.SelectMany((_, i) => CompactCombinationsOfIndexesUptoAndPruneWithTolerance(i, target, target, tolerance, 0ul, 0, k - 1, denoms)));

#0 Sum: 21 -> [10, 10, 1]
#1 Sum: 21 -> [10, 9, 2]
#2 Sum: 21 -> [10, 8, 3]
#3 Sum: 21 -> [10, 6, 5]
#4 Sum: 20 -> [10, 5, 5]
#5 Sum: 21 -> [9, 9, 3]
#6 Sum: 21 -> [9, 8, 4]
#7 Sum: 21 -> [9, 6, 6]
#8 Sum: 21 -> [8, 8, 5]
#9 Sum: 20 -> [8, 6, 6]


In [ ]:
IEnumerable<(long Sum, ulong Decomposition)> cx(int currentDenominationIdx, long target, long localTarget, long tolerance, ulong accumulator, long sum, int k, long[] denoms)
{
	accumulator = (accumulator << 8) | ((ulong)currentDenominationIdx & 0xff);
	var currentDenomination = denoms[currentDenominationIdx];
	sum += currentDenomination;
	var remaining = localTarget - currentDenomination;
	if (k == 0 || remaining < tolerance)
		return new[] { (sum, accumulator) };

	currentDenominationIdx += denoms.Skip(currentDenominationIdx).Count(x => x > remaining);
	return Enumerable.Range(0, denoms.Length - currentDenominationIdx)
		.TakeWhile(i => k * denoms[currentDenominationIdx + i] >= remaining - tolerance)
		.SelectMany((_, i) => 
			cx(currentDenominationIdx + i, target, remaining, tolerance, accumulator, sum, k - 1, denoms)
			.TakeUntil(x => x.Sum == target));
}


Print(denoms.SelectMany((_, i) => cx(i, target, target, tolerance, 0ul, 0, k - 1, denoms)));


#0 Sum: 21 -> [10, 10, 1]
#1 Sum: 21 -> [10, 9, 2]
#2 Sum: 21 -> [10, 8, 3]
#3 Sum: 21 -> [10, 6, 5]
#4 Sum: 20 -> [10, 5, 5]
#5 Sum: 21 -> [9, 9, 3]
#6 Sum: 21 -> [9, 8, 4]
#7 Sum: 21 -> [9, 6, 6]
#8 Sum: 21 -> [8, 8, 5]
#9 Sum: 20 -> [8, 6, 6]


We have now a higher quality result set but the code is a bit hard to follow basically because of the long list of parameters but that can be improved easily. Let start by extracting those that are constant.

In [ ]:
IEnumerable<(long Sum, ulong Decomposition)> cyy(long target, long tolerance, int kk, long[] denoms)
{
	IEnumerable<(long Sum, ulong Decomposition)> cxx(int currentDenominationIdx, ulong accumulator, long sum, int k)
	{
		accumulator = (accumulator << 8) | ((ulong)currentDenominationIdx & 0xff);
		var currentDenomination = denoms[currentDenominationIdx];
		sum += currentDenomination;
		var remaining = target - sum;
		if (k == 0 || remaining < tolerance)
			return new[] { (sum, accumulator) };

		currentDenominationIdx += denoms.Skip(currentDenominationIdx).Count(x => x > remaining);
		return Enumerable.Range(0, denoms.Length - currentDenominationIdx)
			.TakeWhile(i => k * denoms[currentDenominationIdx + i] >= remaining - tolerance)
			.SelectMany((_, i) => 
				cxx(currentDenominationIdx + i, accumulator, sum, k - 1)
				.TakeUntil(x => x.Sum == target));
	}

	return denoms.SelectMany((_, i) => cxx(i, 0ul, 0, kk - 1));
}

Print(cyy(target, tolerance, k, denoms));


#0 Sum: 21 -> [10, 10, 1]
#1 Sum: 21 -> [10, 9, 2]
#2 Sum: 21 -> [10, 8, 3]
#3 Sum: 21 -> [10, 6, 5]
#4 Sum: 20 -> [10, 5, 5]
#5 Sum: 21 -> [9, 9, 3]
#6 Sum: 21 -> [9, 8, 4]
#7 Sum: 21 -> [9, 6, 6]
#8 Sum: 21 -> [8, 8, 5]
#9 Sum: 20 -> [8, 6, 6]
